### Importing Packages

In [1]:
import org.apache.spark.sql.{SparkSession,SQLContext,SaveMode,DataFrame,Row}
import org.apache.spark.sql.types.{StructField, StructType, LongType}
import org.apache.spark.sql.functions.lit
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions._

### Adding MySQL Dependancy

In [2]:
%AddJar https://repo1.maven.org/maven2/mysql/mysql-connector-java/8.0.17/mysql-connector-java-8.0.17.jar

Starting download from https://repo1.maven.org/maven2/mysql/mysql-connector-java/8.0.17/mysql-connector-java-8.0.17.jar
Finished download of mysql-connector-java-8.0.17.jar


### Creating Spark Session

In [3]:
val spark = SparkSession.builder.appName("mysql").getOrCreate()

spark = org.apache.spark.sql.SparkSession@412a01b5


org.apache.spark.sql.SparkSession@412a01b5

### Creating SQL Context

In [4]:
val sqlContext = new SQLContext(sc)

sqlContext = org.apache.spark.sql.SQLContext@2b2af965


org.apache.spark.sql.SQLContext@2b2af965

### MySQL Server Credentials

In [5]:
val url="jdbc:mysql://192.168.2.19:3306/mysql"
val driver = "com.mysql.cj.jdbc.Driver"
//val driver = "com.mysql.jdbc.Driver"
val dbtable = "si_attrib"
val user="root"
val password="Aline-12#"

url = jdbc:mysql://192.168.2.19:3306/mysql
driver = com.mysql.cj.jdbc.Driver
dbtable = si_attrib
user = root
password = Aline-12#


Aline-12#

### Reading File

In [7]:
var filedf = spark.read.format("csv")
                   .options(Map(("header","true"),("inferSchema","true"),("delimiter","\t")))
                   .load("../Resources/SI_attr.tab")

filedf = [Sales_Item_Id: string, Sales_Item_Description: string ... 4 more fields]


[Sales_Item_Id: string, Sales_Item_Description: string ... 4 more fields]

### Show

In [8]:
filedf.show(5)

+------------------+----------------------+-----+-----------------------+----------------+--------------------------------+
|     Sales_Item_Id|Sales_Item_Description|  GIC|Product_Lifecycle_State|Purchasing_Group|Average Standard Production Cost|
+------------------+----------------------+-----+-----------------------+----------------+--------------------------------+
|    IE2:15HP-RFU-7|           7GHZ HP ODU|07599|               OBSOLETE|            null|                             0.0|
|    IE2:15HP-RFU-8|  1500HP RF UNIT, F...|07599|               OBSOLETE|            null|                             0.0|
|IE2:15HP-SHORT-112|    1500HP SHORT, FGHZ|07599|               OBSOLETE|            null|                             0.0|
|IE2:15HP-SHORT-137|     1500P SHORT, FGHZ|07599|               OBSOLETE|            null|                             0.0|
| IE2:15HP-TERM-112|  1500HP 50 OHM TER...|09208|               OBSOLETE|            null|                             0.0|
+-------

### Reading table from MySQL

In [9]:
val tableDF = spark.read
    .format("jdbc")
    .option("url", url)
    .option("driver", driver)
    .option("dbtable", dbtable)
    .option("user", user)
    .option("password", password)
    .load()

tableDF = [id: int, SALES_ITEM_ID: string ... 5 more fields]


[id: int, SALES_ITEM_ID: string ... 5 more fields]

### Checking Schema of the table

In [10]:
tableDF.printSchema

root
 |-- id: integer (nullable = true)
 |-- SALES_ITEM_ID: string (nullable = true)
 |-- SALES_ITEM_DESCRIPTION: string (nullable = true)
 |-- GIC: string (nullable = true)
 |-- PRODUCT_LIFECYCLE_STATE: string (nullable = true)
 |-- PURCHASING_GROUP: string (nullable = true)
 |-- AVERAGESTANDARDPRODUCTIONCOST: double (nullable = true)



### Changing Dataframe columns same as table columns

In [11]:
filedf = filedf.withColumnRenamed("Average Standard Production Cost","AVERAGESTANDARDPRODUCTIONCOST")

filedf = [Sales_Item_Id: string, Sales_Item_Description: string ... 4 more fields]


[Sales_Item_Id: string, Sales_Item_Description: string ... 4 more fields]

### Converting all file dataframe columns to upper case

In [12]:
filedf = filedf.toDF(filedf.columns map (_.toUpperCase): _*)

filedf = [SALES_ITEM_ID: string, SALES_ITEM_DESCRIPTION: string ... 4 more fields]


[SALES_ITEM_ID: string, SALES_ITEM_DESCRIPTION: string ... 4 more fields]

### Generating sequential id column using Window Functions

In [13]:
filedf = filedf.withColumn("id", lit(1))

filedf = [SALES_ITEM_ID: string, SALES_ITEM_DESCRIPTION: string ... 5 more fields]


[SALES_ITEM_ID: string, SALES_ITEM_DESCRIPTION: string ... 5 more fields]

In [14]:
var w = Window.orderBy("id")

w = org.apache.spark.sql.expressions.WindowSpec@5dad8f99


org.apache.spark.sql.expressions.WindowSpec@5dad8f99

In [15]:
filedf = filedf.withColumn("id",row_number.over(w))

filedf = [SALES_ITEM_ID: string, SALES_ITEM_DESCRIPTION: string ... 5 more fields]


[SALES_ITEM_ID: string, SALES_ITEM_DESCRIPTION: string ... 5 more fields]

### Show

In [16]:
filedf.show(5)

+------------------+----------------------+-----+-----------------------+----------------+-----------------------------+---+
|     SALES_ITEM_ID|SALES_ITEM_DESCRIPTION|  GIC|PRODUCT_LIFECYCLE_STATE|PURCHASING_GROUP|AVERAGESTANDARDPRODUCTIONCOST| id|
+------------------+----------------------+-----+-----------------------+----------------+-----------------------------+---+
|    IE2:15HP-RFU-7|           7GHZ HP ODU|07599|               OBSOLETE|            null|                          0.0|  1|
|    IE2:15HP-RFU-8|  1500HP RF UNIT, F...|07599|               OBSOLETE|            null|                          0.0|  2|
|IE2:15HP-SHORT-112|    1500HP SHORT, FGHZ|07599|               OBSOLETE|            null|                          0.0|  3|
|IE2:15HP-SHORT-137|     1500P SHORT, FGHZ|07599|               OBSOLETE|            null|                          0.0|  4|
| IE2:15HP-TERM-112|  1500HP 50 OHM TER...|09208|               OBSOLETE|            null|                          0.0|  5|


### File Dataframe Schema

In [17]:
filedf.printSchema

root
 |-- SALES_ITEM_ID: string (nullable = true)
 |-- SALES_ITEM_DESCRIPTION: string (nullable = true)
 |-- GIC: string (nullable = true)
 |-- PRODUCT_LIFECYCLE_STATE: string (nullable = true)
 |-- PURCHASING_GROUP: string (nullable = true)
 |-- AVERAGESTANDARDPRODUCTIONCOST: double (nullable = true)
 |-- id: integer (nullable = true)



### Checking Table Count before writing

In [18]:
tableDF.count()

0

### Reducing File Count to 10000

In [19]:
filedf = filedf.limit(10000)

filedf = [SALES_ITEM_ID: string, SALES_ITEM_DESCRIPTION: string ... 5 more fields]


[SALES_ITEM_ID: string, SALES_ITEM_DESCRIPTION: string ... 5 more fields]

### Writing to MySQL

In [20]:
filedf.write
      .format("jdbc")
      .options(Map(("url", url),("driver", driver),("dbtable", dbtable),("user", user),("password", password)))
      .mode(SaveMode.Append)
      .save()

### Checking Table Count after writing

In [21]:
val tableDF = spark.read
    .format("jdbc")
    .option("url", url)
    .option("driver", driver)
    .option("dbtable", dbtable)
    .option("user", user)
    .option("password", password)
    .load()

tableDF = [id: int, SALES_ITEM_ID: string ... 5 more fields]


[id: int, SALES_ITEM_ID: string ... 5 more fields]

In [22]:
tableDF.count()

10000

### Closing Spark Session

In [23]:
spark.stop()